# CONFIDENCE 1 

Basic Test, Just to see RTK, Lidar, Code Performance on field 


In [1]:
from IPython.display import HTML
import random




def hide(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'FUNCTIONAL COMPONENT HIDDEN'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

hide()

In [2]:
import math
import numpy as np
from math import radians, cos, sin, asin, sqrt, atan
import geopy
from geopy import distance as GD
from numpy import cross, eye, dot
from scipy.linalg import expm, norm


## C1.1 
Flying 20 metre in front of the windmill blade __bottom facing__

In [3]:
# INPUTS 

# GPS Coordinates of the lidar 
lat0 = 11.18138416
lon0 = 77.29683663

# GPS Coordinate of known point  # Point 1 - RTK 
lat1 = 11.181465059
lon1 = 77.296846757
 
# Point 1 ->  Coordinates (x,y) INITIAL CALIBRATION POINT - RTK 
vA = [60.5578,-9.75103,-2.1644]

# Point 2 -> Coordinates (x,y) TARGET POINT 
vB = [53.882,-13.671,-1.917]


# Elevation of lidar from ground 
theta = math.radians(-41)

# Lidar Drone Offset
lidar_drone_offset = 1.0

x = vB[0]
y = vB[1]



In [4]:
# FUNCTIONS


def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    
    
    bearing = math.atan2(sin(lon2-lon1)*cos(lat2), cos(lat1)*sin(lat2)-sin(lat1)*cos(lat2)*cos(lon2-lon1))
    bearing = math.degrees(bearing)
    bearing = (bearing + 360) % 360

    
    
    return c * r , bearing


def dot(vA, vB):
    return vA[0]*vB[0]+vA[1]*vB[1]

def ang(vA, vB):
    # Get dot prod
    dot_prod = dot(vA, vB)
    # Get magnitudes
    magA = dot(vA, vA)**0.5
    magB = dot(vB, vB)**0.5
    # Get cosine value
    cos_ = dot_prod/magA/magB
    # Get angle in radians and then convert to degrees
    angle = math.acos(dot_prod/magB/magA)
    # Basically doing angle <- angle mod 360
    ang_deg = math.degrees(angle)%360

    if ang_deg-180>=0:
        # As in if statement
        return 360 - ang_deg
    else: 

        return ang_deg


def get_distance(x,y):
    return math.sqrt((x**2+y**2))/1000

import geopy

def get_coord_2(bearing,lat,lon,dist):
    b = bearing
    d = dist
    lat1 = lat 
    lon1 = lon
    origin = geopy.Point(lat1, lon1)
    for model_name in models :
        destination = GD.geodesic(kilometers=d,ellipsoid=model_name).destination(origin, b,)
        lat2, lon2 = destination.latitude, destination.longitude
        print(model_name,' \t: ',lat2,lon2)
        
        
ELLIPSOIDS = {'WGS-84':        (6378.137,    6356.7523142,  1 / 298.257223563),
              'GRS-80':        (6378.137,    6356.7523141,  1 / 298.257222101),
              'Airy (1830)':   (6377.563396, 6356.256909,   1 / 299.3249646),
              'Intl 1924':     (6378.388,    6356.911946,   1 / 297.0),
              'Clarke (1880)': (6378.249145, 6356.51486955, 1 / 293.465),
              'GRS-67':        (6378.1600,   6356.774719,   1 / 298.25),
              }

models = list(ELLIPSOIDS.keys())

def get_coord_1(bearing,lat,lon,dist,Rmodel='dji'):
    brng = math.radians(bearing)
    
    d = dist 
    if Rmodel=='dji' :
        R = 6378.137
    elif Rmodel=='google':
        R = 6371.0710
    lat1 = math.radians(lat)
    lon1 = math.radians(lon) 
    lat2 = math.asin( math.sin(lat1)*math.cos(d/R) + math.cos(lat1)*math.sin(d/R)*math.cos(brng))
    lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1),math.cos(d/R)-math.sin(lat1)*math.sin(lat2))
    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)
    
    print (Rmodel,' \t\t', lat2,lon2)

hide()

In [5]:
lat = [11.12324352,11.12324477,11.12324479,11.12324419]
long = [77.32849404,77.32849421,77.32849347,77.32849323]

In [13]:
test = 1
test2 = 3
lat[test],long[test]
lat[test2],long[test2]

haversine(long[test], lat[test], long[test2], lat[test2])

(0.00012486827224352404, 238.9029831300757)

### LONGLAT CALCULATION 

In [5]:
distacePointRtk, bearing = haversine(lon0, lat0, lon1, lat1)


In [10]:
bearing

122.18277304431444

In [6]:
offset = ang(vA,vB) # Offset of p1 from p2 


In [11]:
offset

5.08946595886398

In [13]:
print(vA,vB)

[60.5578, -9.75103, -2.1644] [53.882, -13.671, -1.917]


In [7]:
# Options 
bPlus = bearing + offset
bNeg = bearing - offset

In [8]:
print('POSITIVE DISTANCE OPTION')
get_coord_2(bearing=bPlus,lat=lat0,lon=lon0,dist=get_distance(x,y))

print('\nNEGATIVE DISTANCE OPTION')
get_coord_2(bearing=bNeg,lat=lat0,lon=lon0,dist=get_distance(x,y))

POSITIVE DISTANCE OPTION
WGS-84  	:  10.897255892834067 77.26459052847241
GRS-80  	:  10.897255892834062 77.26459052847241
Airy (1830)  	:  10.897255872376695 77.26459056503748
Intl 1924  	:  10.897255896604404 77.26459051234502
Clarke (1880)  	:  10.897255866525697 77.26459052056877
GRS-67  	:  10.89725589388461 77.26459052701213

NEGATIVE DISTANCE OPTION
WGS-84  	:  10.89733135696302 77.26463857907149
GRS-80  	:  10.897331356963013 77.2646385790715
Airy (1830)  	:  10.897331341578123 77.26463861997877
Intl 1924  	:  10.897331359798494 77.26463856102895
Clarke (1880)  	:  10.897331337177933 77.26463857022927
GRS-67  	:  10.897331357753076 77.2646385774378


### ALTITUDE CALCULATION

In [34]:
# INPUT 
target = np.array([[vB[0]],[vB[1]],[vB[2]],[1]])

In [35]:
# Translation Matrix in the Y Direction 
tY = np.array([[math.cos(theta),0,math.sin(theta),0],
          [0,1,0,0],
          [-1*math.sin(theta),0,math.cos(theta),0],
          [0,0,0,1]
         ])

res = tY @ target
hide()

In [36]:
# OUTPUT 
height = res[2]

In [37]:
height = height - lidar_drone_offset

In [38]:
height

array([34.49562897])